# 🦅 AVCI - Yüksek Oran Avcısı Eğitim Paneli v0.1.1 (Standalone)
**Akıllı Çözüm:**
- **Smart DB Loader:** Veritabanı bozuksa veya tablo yoksa, sistem bunu algılar ve **sadece o zaman** Drive'dan yedeği çeker.
- Normal şartlarda Drive en sonda bağlanır.
- **1000x Modu:** Aktif.
---

In [ ]:
# 1. GitHub Deposunu Çek ve Kurulum Yap (Sessiz Kurulum)
import os
import sys
import shutil

print("🚀 Kurulum Başlatılıyor...")

# 1. Kök dizine (Colab varsayılanı) dön
try:
    os.chdir('/content')
except:
    pass

GITHUB_REPO = "https://github.com/onndd/avci.git"
PROJECT_DIR = "avci"

# 2. Varsa eski projeyi değil, sadece bu projeyi kontrol et
if not os.path.exists(PROJECT_DIR):
    print(f"📥 Proje indiriliyor: {GITHUB_REPO}")
    !git clone $GITHUB_REPO
else:
    print("✅ Proje klasörü mevcut. Güncelleniyor...")
    os.chdir(PROJECT_DIR)
    !git pull
    os.chdir('..')

# 3. Çalışma dizinine gir ve YOLU EKLE
if os.path.exists(PROJECT_DIR):
    os.chdir(PROJECT_DIR)
    if os.getcwd() not in sys.path:
        sys.path.append(os.getcwd())
    print(f"✅ Çalışma Dizini Ayarlandı: {os.getcwd()}")
else:
    print(f"❌ HATA: {PROJECT_DIR} klasörü bulunamadı!")

# 4. Dosyaları listele (Hata ayıklama için)
print("📂 Klasör İçeriği:")
!ls -F

!nvidia-smi
!pip install optuna streamlit matplotlib pandas tensorflow hmmlearn plotly lightgbm
print("✅ Kurulum ve Hazırlık Tamamlandı!")

In [ ]:
# 2. Veri Yükleme ve Ayarlar
import optuna
import logging
import sys
import os

optuna.logging.set_verbosity(optuna.logging.INFO)
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))

# Mevcut dizini path'e ekle (Garanti olsun)
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

try:
    import train_avci
    import models_avci
    import model_ae_avci
    import model_hmm_avci
    load_and_prep = train_avci.load_and_prep
    visualize_performance = train_avci.visualize_performance
    optimize_target = train_avci.optimize_target
    train_target_final = train_avci.train_target_final
    print("✅ Modüller başarıyla yüklendi!")
except ImportError as e:
    print(f"❌ KRİTİK HATA: Modüller yüklenemedi. Detay: {e}")
    print("Lütfen 'Setup' hücresini (yukarıdaki) çalıştırdığınızdan emin olun.")
    print(f"Şu anki dizin: {os.getcwd()}")

# Veri Yükleme
if 'load_and_prep' in locals():
    print("📂 Veri yükleniyor...")
    try:
        df = load_and_prep(limit=100000)
        print(f"✅ Veri Hazır: {len(df)} satır.")
    except Exception as e:
        print(f"⚠️ Veri yükleme hatası: {e}")


## 🕵️‍♂️ Bölüm 1: İstihbarat & Komuta

In [ ]:
if 'model_ae_avci' in locals():
    ae = model_ae_avci.train_autoencoder(df)

In [ ]:
if 'model_hmm_avci' in locals():
    hmm_model = model_hmm_avci.train_hmm(df)

## 🎯 Bölüm 2: Nişancı Eğitimi (Adım Adım)

In [ ]:
# 1.50x - OPTİMİZASYON 🛠️
TARGET = 1.5
EPOCHS = 30
study_15, params_15 = optimize_target(df, TARGET, epochs=EPOCHS)

In [ ]:
# 1.50x - FİNAL EĞİTİM 🏃
model_15, _, _ = train_target_final(df, TARGET, params_15)
split_idx = int(len(df) * 0.85)
y_col = f'target_{str(TARGET).replace(".","_")}'
X_val = df.iloc[split_idx:][[c for c in df.columns if 'target' not in c and 'result' not in c and 'value' not in c and 'id' not in c]]
y_val = df[y_col].iloc[split_idx:]
visualize_performance(model_15, X_val, y_val, TARGET)

In [ ]:
# 3.00x - OPTİMİZASYON 🛠️
TARGET = 3.0
EPOCHS = 30
study_30, params_30 = optimize_target(df, TARGET, epochs=EPOCHS)

In [ ]:
# 3.00x - FİNAL EĞİTİM 🏃
model_30, _, _ = train_target_final(df, TARGET, params_30)
y_val = df[f'target_{str(TARGET).replace(".","_")}'].iloc[split_idx:]
visualize_performance(model_30, X_val, y_val, TARGET)

In [ ]:
# 5.00x - OPTİMİZASYON 🛠️
TARGET = 5.0
EPOCHS = 30
study_50, params_50 = optimize_target(df, TARGET, epochs=EPOCHS)

In [ ]:
# 5.00x - FİNAL EĞİTİM 🏃
model_50, _, _ = train_target_final(df, TARGET, params_50)
y_val = df[f'target_{str(TARGET).replace(".","_")}'].iloc[split_idx:]
visualize_performance(model_50, X_val, y_val, TARGET)

In [ ]:
# 10.0x - OPTİMİZASYON 🛠️
TARGET = 10.0
EPOCHS = 30
study_100, params_100 = optimize_target(df, TARGET, epochs=EPOCHS)

In [ ]:
# 10.0x - FİNAL EĞİTİM 🏃
model_100, _, _ = train_target_final(df, TARGET, params_100)
y_val = df[f'target_{str(TARGET).replace(".","_")}'].iloc[split_idx:]
visualize_performance(model_100, X_val, y_val, TARGET)

In [ ]:
# 100.0x - OPTİMİZASYON (Avcı Modu) 🛠️
TARGET = 100.0
EPOCHS = 50
study_1000, params_1000 = optimize_target(df, TARGET, epochs=EPOCHS)

In [ ]:
# 100.0x - FİNAL EĞİTİM 🏃
model_1000, _, _ = train_target_final(df, TARGET, params_1000)
y_val = df[f'target_{str(TARGET).replace(".","_")}'].iloc[split_idx:]
visualize_performance(model_1000, X_val, y_val, TARGET)

In [ ]:
# 1000.0x - OPTİMİZASYON (God Mode) 🛠️
TARGET = 1000.0
EPOCHS = 50
study_god, params_god = optimize_target(df, TARGET, epochs=EPOCHS)

In [ ]:
# 1000.0x - FİNAL EĞİTİM 🏃
model_god, _, _ = train_target_final(df, TARGET, params_god)
y_val = df[f'target_{str(TARGET).replace(".","_")}'].iloc[split_idx:]
visualize_performance(model_god, X_val, y_val, TARGET)

## 💾 Bölüm 3: Modelleri Kaydet ve İndir

In [ ]:
# Tüm Modelleri Ziple ve Drive'a Gönder
import os
if not os.path.exists('models'):
    os.makedirs('models')
    
!zip -r avci_models.zip models/

# Drive Mount (Sadece Şimdi)
try:
    from google.colab import drive
    drive.mount('/content/drive')
    
    !cp avci_models.zip /content/drive/MyDrive/avci_backup_latest.zip
    print("✅ Yedek Drive'a (Ana Dizin) başarıyla kopyalandı.")
except Exception as e:
    print(f"⚠️ Drive'a kopyalanamadı: {e}")

# İndir (Opsiyonel)
try:
    from google.colab import files
    files.download('avci_models.zip')
except Exception as e:
    pass